# Introduction

Teure Uhren, die man sich als Student nicht leisten kann, sind kein tolles Hobby. 
Wenn es allerdings eine Möglichkeit gäbe herauszufinden welche Uhr sehr teuer aussieht es aber nicht ist. 
So wäre es Möglich als armer Student trotzdem prollen zu können. 
Da das Aussehen einer Uhr besonders Subjektiv ist, muss eine Objektive Möglichkeit gefunden werden um den Preis von Uhren anhand des Aussehens zu schätzen.
Hier kommt Machine Learning ins Spiel. Ein Modell soll anhand von Mathematischen Grundlagen herausfinden, welche Uhren teuerer und welche billiger aussehen. 

Aus dieser Herrausforderung ensteht das folgende Projekt.
Das konkrete Ziel ist es:
**Den Preis einer Uhr anhand eines Bildes der Uhr vorherzusagen.**

# Data

Zur Lösung des Problems wurden drei Datensätze zusammengestellt und gedowloaded.
Der erste Datensatz ist ein offizieller Datensatz von Kaggle, welcher frei zugänglich ist. 

## DS_1 [Kaggle - A Dataset of Watches](https://www.kaggle.com/datasets/mathewkouch/a-dataset-of-watches)

Dieser Datensatz umfasst 2553 Uhrenbilder und zusätzliche Informationen zur Marke zum Namen der Uhr und zum Preis in US Dollar. Zum Zeitpunkt der Arbeit gibt es keine öffentlichen Codebeispiele für den Datensatz.

## DS_2 Scraped OnlineShop [Marc Gebauer](https://marcgebauer.com/collections/alle-luxus-uhren)

Mithilfe eines [GITHub Shopify Scraper](https://github.com/lagenar/shopify-scraper) der minimal angepasst wurde, wurden 230 Uhrenbilder vom Marc Gebauer Shop gescraped. Zusätzlich konnte der Scraper auch die Preise der Produkte festhalten und in einer CSV abspeichern.

## DS_3 Scraped OnlineShop [Crown & Caliber](https://www.crownandcaliber.com/)
Ebenfalls mithilfe es angepassten [GITHub Shopify Scraper](https://github.com/lagenar/shopify-scraper) konnten in diesem OnlineShop 26.216 Bilder Uhren und deren Preis gescraped werden. 

In [19]:
### Imports for this Notebook
import matplotlib as plt
import pandas as pd

import urllib.request 
import pandas as pd
import concurrent.futures

# Scraper

### Scrape Products from Shops with [GitHub Scraper](https://github.com/lagenar/shopify-scraper)

In [ ]:
from shopify_scraper import extract_products

In [16]:
### TIME INFORMATION: needs 1,5 Minutes to run
extract_products("https://marcgebauer.com", 'data/metadata/marc_products.csv')

In [ ]:
### TIME INFORMATION: needs X,X Minutes to run
extract_products("https://www.crownandcaliber.com", 'data/metadata/crown_products.csv')

### Download images from the shops

In [20]:
def get_picture(row):
    url = row[1]["Image URL"]
    code = row[1]["Code"]
    
    try:
        # Bild herunterladen und speichern
        image_filename = "data/watches/" + folder + f"/newImages/{code}.jpg"
        urllib.request.urlretrieve(url, image_filename)
        return "Bild gespeichert:" + image_filename
    except Exception as e:
        return "Fehler beim Herunterladen des Bildes:" +  url +  "Fehlermeldung:" +  str(e)
    
def download_images(df):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:

        future_row = [executor.submit(get_picture, row) for row in df.iterrows()]

        for future in concurrent.futures.as_completed(future_row):
            try:
                print(future.result())
            except Exception as exc:
                print(f"Fehler: {exc}")

In [ ]:
df_marc = pd.read_csv("data/metadata/marc_products.csv")

## Get Shop Items

## Get Images

# Data Preperation

In [3]:
## Helperfunctions

### Change the used filetype here
def add_jpg_suffix(string):
    return string + '.jpg'

### You can change the conversion rate here
def convert_usd_to_eur(price):
    return price * 1.0675

In [4]:
def format_data(df, drop_colums = None,
                      replace_stringparts = None, 
                      convert_price = None,
                      covert_to_usd = False,
                      select_category= None,
                      create_path_from_name = False,
                      reindex = None):

    #Choose a category from the Shopify Shop Categories if the Shop has not only watches
    if select_category != None:
        df[df['Category'] == select_category]
    
    #Drop the columns we don't need for the Model
    if drop_colums != None:
        df = df.drop(drop_colums, axis=1)
    
    #Replace unnecessary string parts in the Price String of the Kaggle Data
    if replace_stringparts != None:
        for replace_object in replace_objects:
            df['price'] = df['price'].replace(replace_object , '', regex=True)
    
    #Convert the Price to a specific Datatype
    if convert_price != None:
        df["price"].astype(float)
        
    if convert_to_usd:
        df['price'] = df['price'].apply(convert_usd_to_eur)
        
    if create_path_from_name:
        df['full_path'] = df['Code'].apply(add_jpg_suffix)
        
    if reindex != None:
        df = df.reindex(columns=reindex)

    return df

## Clean DS2 Marc Gebauer Products

In [ ]:
df_marc = pd.read_csv("data/metadata/marc_products.csv")
df_marc = format_data(df_marc,
                        create_path_from_name=True,
                        reindex=['full_path', 'price'],
                        drop_colums=['Name', 'URL', 'Image URL','Collection', 'Category', 'Variant Name', 'In Stock'])
df_marc.to_csv("data/watches/ds2_marc/cleandata.csv")
df_marc.head()

## Clean DS3 Crown Caliber Products

In [10]:
df_crown = pd.read_csv("data/metadata/crown_products.csv")
df_crown = format_data(df_crown,
                        create_path_from_name=True,
                        reindex=['full_path', 'price'],
                        drop_colums=['Name', 'URL', 'Image URL','Collection', 'Category', 'Variant Name', 'In Stock'])
df_crown.to_csv("data/watches/ds3_crown/cleandata.csv")
df_crown.head()

# First Data Inspection